# Python notebook 

## In memory approach as opposed to a database
Amazon co-purchase data analysis


In [27]:
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import pandas as pd

In [28]:
file = 'C:/Users/moore/Dropbox/graphs/Amazon0601.txt'
location_gml = 'C:/Users/moore/Dropbox/graphs/Amazon.gml'
location_pickle = 'C:/Users/moore/Dropbox/graphs/Amazon.gpickle'
file_meta = 'C:/Users/moore/Dropbox/graphs/Amazon-meta.txt'
pickle_meta = 'C:/Users/moore/Dropbox/graphs/AmazonMeta.p'
location_pairs = 'C:/Users/moore/Dropbox/graphs/Amazon.p'

review_csv = 'C:/Users/moore/Dropbox/graphs/reviews.csv'
sims_csv = 'C:/Users/moore/Dropbox/graphs/sims.csv'
prods_csv = 'C:/Users/moore/Dropbox/graphs/prods.csv'
bought_csv = 'C:/Users/moore/Dropbox/graphs/bought.csv'
cats_csv = 'C:/Users/moore/Dropbox/graphs/cats.csv'
prods_csv = 'C:/Users/moore/Dropbox/graphs/prods_dataframe.csv'
product_details = 'C:/Users/moore/Dropbox/graphs/prods_data.p'
pair_listing = 'C:/Users/moore/Dropbox/graphs/pair_list.p'
pair_dictionary = 'C:/Users/moore/Dropbox/graphs/pair_dict.p'

In [29]:
def in_memory():
    restructured = pickle.load(open(product_details, 'rb'))
    pair_dict = pickle.load(open(pair_dictionary, 'rb'))
    paths = pickle.load(open(pair_listing, 'rb')) 
    records = []
    for path in paths:
        dic = {}
        pairs = (path[0],path[1])
        try:
            a = pair_dict[pairs[1]]
        except:
            a = 0
        dic = {'Title': restructured[pairs[0]]['title'], 
           'co_purNum': pair_dict[pairs[0]], 
           'pair Title': restructured[pairs[1]]['title'], 
           'pair_Count': a}
        records.append(dic)
    
    df = pd.DataFrame.from_records(records)
    return df.shape

In [30]:
def neo4j():
    from neo4jrestclient.client import GraphDatabase
    from neo4jrestclient.constants import RAW
    from neo4jrestclient.client import Node 
    q = """MATCH p=(n)-[r:co_pur]->(w) RETURN n.title, n.id, n.cats, n.status, w.title, w.id, w.cats, w.status"""
    db = GraphDatabase("http://localhost:7474/db/data")
    #retrieve a list of all nodes with the labels
    params = {}
    results = db.query(q, params=params, returns=RAW)
    records = []
    for prod in results:
        records.append(prod)
    
    cols = ['n.title', 'n.id', 'n.cats', 'n.status', 'w.title', 'w.id', 'w.cats', 'w.status']
    df = pd.DataFrame.from_records(records, columns=cols)  
    return df.shape

In [31]:
%%time
shape=in_memory()
print(shape, 'returned from in-memory approach ')

(3387388, 4) returned from in-memory approach 
Wall time: 13 s


In [32]:
%%time
shape=neo4j()
print(shape, 'returned from Cypher query on local Neo4j database')

(3387388, 8) returned from Cypher query on local Neo4j database
Wall time: 57.8 s
